In [2]:
import syft as sy
import torch as t
import numpy as np
import pandas as pd

## Launch a Duet Sever

In [2]:
duet = sy.launch_duet(loopback = True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...

♫♫♫ > CONNECTED!



In [3]:
duet = sy.launch_duet()

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!
♫♫♫ > Duet Server ID: f246064f24d360c68e7b410770dcc17d

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.duet("f246064f24d360c68e7b410770dcc17d")

♫♫♫ > STEP 2: Ask your partner for their Client ID and enter it below!
♫♫♫ > Duet Partner's Client ID: 7b065fe7f5cbdd5e7a3a0221c36b5b35

♫♫♫ > Connecting...

♫♫♫ > CONNECTED!



## Read Data and Preprocess

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

german = pd.read_csv("./data/german_data_1.csv")  

In [4]:
test_size = 0.20

processed_data = None
categorical = None
label_encoders = {}

def preprocessing(dataset, data, test_size):
    """
    Preprocess dataset

    Parameters
    ----------
    data: DataFrame
        Pandas dataframe containing German dataset.
    """
    
    global processed_data
    global categorical
    global label_encoders

    # Reset global variables
    
    processed_data = None
    categorical = None
    label_encoders = {}


    if dataset == "German":
        # Drop savings account and checkings account columns as they contain a lot
        # of NaN values and may not always be available in real life scenarios
        data = data.drop(columns = ['Saving accounts', 'Checking account'])
        
    dat_dict = data.to_dict()
    new_dat_dict = {}

    # rename columns(Make them lowercase and snakecase)
    for key, value in dat_dict.items():
        newKey = key
        if type(key) == str:
            newKey = newKey.lower().replace(' ', '_')
        # if newKey != key:
        new_dat_dict[newKey] = dat_dict[key]
    del dat_dict

    data = pd.DataFrame.from_dict(new_dat_dict)
    del new_dat_dict


    # print(data.describe())
    # print(data.describe(include='O'))

    cols = data.columns
    num_cols = data._get_numeric_data().columns
    categorical = list(set(cols) - set(num_cols))

    # Drop null rows
    data = data.dropna()

    # Encode text columns to number values
    for category in categorical:
        le = LabelEncoder()
        data[category] = le.fit_transform(data[category])
        label_encoders[category] = le

    for col in data.columns:
        if(col not in categorical):
            data[col] = (data[col].astype('float') - np.mean(data[col].astype('float')))/np.std(data[col].astype('float'))

    # print(data.describe())
    # print(data.describe(include='O'))

    processed_data = data

    # Get Training parameters
    if dataset == "German":
        target_col = data.columns[-1]
        x = data.drop(columns=target_col, axis=1)
        y = data[target_col].astype('int')
    elif dataset == "Australian":
        x = data.drop(14, axis=1)
        y = data[14].astype('int')
    elif dataset == "Japanese":
        x = data.drop(15, axis=1)
        y = data[15].astype('int')
    elif dataset == "Taiwan":
        x = data.drop('default_payment_next_month', axis=1)
        y = data['default_payment_next_month'].astype('int')
    elif dataset == "Polish":
        x = data.drop('class', axis=1)
        y = data['class'].astype('int')


    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size)
    x_train = pd.DataFrame(x_train)
    y_train = pd.DataFrame(y_train)
    y_test = pd.DataFrame(y_test)

    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    
    y_train = y_train[y_train.columns[0]].to_numpy()
    y_test = y_test[y_test.columns[0]].to_numpy()

    return (x_train, x_test, y_train, y_test)

X_train, X_test, y_train, y_test = preprocessing("German", german, test_size)

In [5]:
# Trnasform the input to tensor
X_train_tensor = t.FloatTensor(X_train)
y_train_tensor = t.tensor(y_train,dtype=t.long)
X_test_tensor = t.FloatTensor(X_test)
y_test_tensor = t.tensor(y_test,dtype=t.long)

## Send data to Duet server

In [6]:
X_train_tensor.tag("X_Train_Owner1")
X_train_tensor.describe("Dataset of 400 samples, 7 feature")
X_train_ptr = X_train_tensor.send(duet,pointable = True)

y_train_tensor.tag("y_Train_Owner1")
y_train_tensor.describe("Dataset of 400 samples, 1 feature")
y_train_ptr = y_train_tensor.send(duet,pointable = True)

X_test_tensor.tag("X_Test_Owner1")
X_test_tensor.describe("Dataset of 400 samples, 7 feature")
X_test_ptr = X_test_tensor.send(duet,pointable = True)

y_test_tensor.tag("y_Test_Owner1")
y_test_tensor.describe("Dataset of 400 samples, 1 feature")
y_test_ptr = y_test_tensor.send(duet,pointable = True)


## Create a request handler

In [7]:
duet.requests.add_handler(
    action="accept",
    print_local=True,  # print the result in your notebook
)

In [8]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 525378a8d9624a9e886c8b2f3068baac>,[X_Train_Owner1],"Dataset of 400 samples, 7 feature",<class 'torch.Tensor'>
1,<UID: 854a07c3bf394e44b5625c16364f9755>,[y_Train_Owner1],"Dataset of 400 samples, 1 feature",<class 'torch.Tensor'>
2,<UID: f265f31823664e1788e931aadd2b2dbc>,[X_Test_Owner1],"Dataset of 400 samples, 7 feature",<class 'torch.Tensor'>
3,<UID: 58deaf7d9f6140f084971563bd5da3e2>,[y_Test_Owner1],"Dataset of 400 samples, 1 feature",<class 'torch.Tensor'>


[2022-07-01T08:26:06.949550-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: e48cb2c2c68942d9b983e65de978ad4b>: To evaluate training progress
Value: <Storable: torch.Size([400, 7])>
[2022-07-01T08:26:40.819399-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 76dafb488d7a44a4b2ecf90185153eee>: To evaluate training progress
Value: <Storable: 0.7004346251487732>
[2022-07-01T08:26:41.719768-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: da30d2e2bde748bca5bf3fd8d219cdc8>: To evaluate training progress
Value: <Storable: 0.6989440321922302>
[2022-07-01T08:26:42.819692-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 33509d74d1ba498ba980744ad2734488>: To evaluate training progress
Value: <Storable: 0.6951220035552979>
[2022-07-01T08:26:42.996985-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 27a70ed535524dbabae387f80ef057ce>: To evaluate training progress
Value: <Storable: 0.695527195930481>
[2022-07-01T08:26:43.184647-0400][CRITICAL][logger]][37660] > HA

[2022-07-01T08:26:50.931694-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 375ec1266e354d5e8a3c2f97a94e309b>: To evaluate training progress
Value: <Storable: 0.6533764004707336>
[2022-07-01T08:26:51.109076-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: d6b8ecfdc7ad49f5afa3aec47c21b0c3>: To evaluate training progress
Value: <Storable: 0.6485192775726318>
[2022-07-01T08:26:51.285797-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 1bbafd66b2b24abebec14e86857cd6ed>: To evaluate training progress
Value: <Storable: 0.6508879065513611>
[2022-07-01T08:26:51.449485-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 700766f191b9492f866c41b26de7093a>: To evaluate training progress
Value: <Storable: 0.647845983505249>
[2022-07-01T08:26:51.619387-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 2f3eb648c0e743da8dea21db11d0a319>: To evaluate training progress
Value: <Storable: 0.6469880938529968>
[2022-07-01T08:26:51.798919-0400][CRITICAL][logger]][37660] > HAND

[2022-07-01T08:27:01.509036-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 0bd804d7daf74adaab963e5f6ebdcef7>: To evaluate training progress
Value: <Storable: 0.6137800812721252>
[2022-07-01T08:27:01.677025-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 2c35187d658e498eb643de6a8fd177f1>: To evaluate training progress
Value: <Storable: 0.6134483218193054>
[2022-07-01T08:27:01.851687-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 4deafd1b4c2048818d30de21e6f577f8>: To evaluate training progress
Value: <Storable: 0.6159021854400635>
[2022-07-01T08:27:02.029499-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 669e5fc9fff048faae2a1f6bec3e60a6>: To evaluate training progress
Value: <Storable: 0.6153659820556641>
[2022-07-01T08:27:02.204866-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: c162ab65c48543eba2745ee046b0b758>: To evaluate training progress
Value: <Storable: 0.6168888211250305>
[2022-07-01T08:27:02.379479-0400][CRITICAL][logger]][37660] > HAN

[2022-07-01T08:27:10.136894-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 07b2e06ea63b40eda731afe8407d480f>: To evaluate training progress
Value: <Storable: 0.580796480178833>
[2022-07-01T08:27:10.290151-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: f7523a5c47d24507a25ea2cea66e05d0>: To evaluate training progress
Value: <Storable: 0.5819178223609924>
[2022-07-01T08:27:10.454959-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: b7f1f550578f43a9b85d488425d8bec4>: To evaluate training progress
Value: <Storable: 0.5799546241760254>
[2022-07-01T08:27:10.626750-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: ecae695c668241b69bec1a08b45200ec>: To evaluate training progress
Value: <Storable: 0.5795045495033264>
[2022-07-01T08:27:10.799208-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: cad33b73b4584cc7952cc909f43efe21>: To evaluate training progress
Value: <Storable: 0.5762068629264832>
[2022-07-01T08:27:10.976218-0400][CRITICAL][logger]][37660] > HAND

...

[2022-07-01T08:29:13.179111-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 79e4b83db69d4533b5ed6133e0e80026>: To evaluate training progress
Value: <Storable: 0.4281628131866455>
[2022-07-01T08:29:13.328996-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 24695d64a11241f6b83423aef9731f30>: To evaluate training progress
Value: <Storable: 0.4213966429233551>
[2022-07-01T08:29:13.486419-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 277302b63cd44281a06403f46d4bdff0>: To evaluate training progress
Value: <Storable: 0.43225201964378357>
[2022-07-01T08:29:13.646413-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 30be32d9c7d142949945d60e28a98e6c>: To evaluate training progress
Value: <Storable: 0.4281265139579773>
[2022-07-01T08:29:13.810237-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 7f3707c7474a4e41800b04ad6b3069c8>: To evaluate training progress
Value: <Storable: 0.42642661929130554>
[2022-07-01T08:29:13.967993-0400][CRITICAL][logger]][37660] > H

[2022-07-01T08:29:22.080068-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 2d92589eb85a4ef4a867abf0b5c78429>: To evaluate training progress
Value: <Storable: 0.4253791570663452>
[2022-07-01T08:29:22.251868-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 7b4466cfa64749ada5436212ec9c8c7c>: To evaluate training progress
Value: <Storable: 0.43071311712265015>
[2022-07-01T08:29:22.419040-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: f382f2a22e4e4b5a8f66e00e6c84794a>: To evaluate training progress
Value: <Storable: 0.4271719753742218>
[2022-07-01T08:29:22.566372-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: a27fe37ebc8e482486286006b5b0dd6f>: To evaluate training progress
Value: <Storable: 0.4199371039867401>
[2022-07-01T08:29:22.728409-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: dc27b37a9aaa4061be32b77b9eb56337>: To evaluate training progress
Value: <Storable: 0.4280402660369873>
[2022-07-01T08:29:22.879114-0400][CRITICAL][logger]][37660] > HA

[2022-07-01T08:29:29.974239-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: e6551ecd84914045afb9b471bd07943b>: To evaluate training progress
Value: <Storable: 0.4274047911167145>
[2022-07-01T08:29:30.132345-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: a13bf12cfc1b47f68455cb33d463136b>: To evaluate training progress
Value: <Storable: 0.42339423298835754>
[2022-07-01T08:29:30.299093-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 26667afac55b4e0facb119586d9962ce>: To evaluate training progress
Value: <Storable: 0.41833531856536865>
[2022-07-01T08:29:30.454499-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: f2edb0259d504ba091590f978f331390>: To evaluate training progress
Value: <Storable: 0.43152427673339844>
[2022-07-01T08:29:31.588346-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: ee17e956ecb34ce6b464ba1958fcb104>: To evaluate training progress
Value: <Storable: 0.426039457321167>
[2022-07-01T08:29:31.742397-0400][CRITICAL][logger]][37660] > H

[2022-07-01T08:29:38.798427-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: d9c779b6a0124e07b04a4cefe5011aa6>: To evaluate training progress
Value: <Storable: 0.4199462831020355>
[2022-07-01T08:29:38.948232-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 4362c83ca9754f6a8278b5188a9eff7a>: To evaluate training progress
Value: <Storable: 0.42224016785621643>
[2022-07-01T08:29:39.118949-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: f06e5c5357e34a0ca8f66024c1177f91>: To evaluate training progress
Value: <Storable: 0.42226603627204895>
[2022-07-01T08:29:39.278932-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 3fcf01d6a148484d98af5ea1d3d5900a>: To evaluate training progress
Value: <Storable: 0.4300926923751831>
[2022-07-01T08:29:39.440092-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: f490eb3ac0db41ab840fbc7bf1ead224>: To evaluate training progress
Value: <Storable: 0.4198923110961914>
[2022-07-01T08:29:39.594375-0400][CRITICAL][logger]][37660] > H

[2022-07-01T08:29:46.672678-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 9553442f5f8b4625b631416236e7509c>: To evaluate training progress
Value: <Storable: 0.42170771956443787>
[2022-07-01T08:29:46.826339-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 4206378fba184446afb1d2b7be879009>: To evaluate training progress
Value: <Storable: 0.4214991331100464>
[2022-07-01T08:29:46.986279-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 61f6802fd5384b9ba82666a8bb1fc6fe>: To evaluate training progress
Value: <Storable: 0.424566388130188>
[2022-07-01T08:29:47.159102-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 65653a3e0a5b45c1b4e8cbab644f568b>: To evaluate training progress
Value: <Storable: 0.4184213876724243>
[2022-07-01T08:29:47.298488-0400][CRITICAL][logger]][37660] > HANDLER Request <UID: 52ebe08687614dbf8225f78087ec38e1>: To evaluate training progress
Value: <Storable: 0.4235820472240448>
[2022-07-01T08:29:47.458868-0400][CRITICAL][logger]][37660] > HAN

## Build a local Classifier

In [161]:
class Net(t.nn.Module):
    def __init__(self,input_channels,output_channels):
        super(Net,self).__init__()
        #Our network:
        # Linear1->relu->Batchnorm->Linear2->relu->Batchnorm->Dropout->Linear3->output
        # Softmax is added in the predict function
        #This applies Linear transformation to input data. 
        self.fc1 = t.nn.Linear(input_channels,int(1.5*input_channels))
        self.fc2 = t.nn.Linear(int(1.5*input_channels),int(1.5*input_channels))
        self.fc3 = t.nn.Linear(int(1.5*input_channels),output_channels)
        
        self.relu = t.nn.ReLU()
        self.dropout = t.nn.Dropout(p=0.1)
        self.batchnorm1 = t.nn.BatchNorm1d(int(1.5*input_channels))
        self.batchnorm2 = t.nn.BatchNorm1d(int(1.5*input_channels))
        self.sigmoid = t.nn.Sigmoid()
    #This must be implemented
    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.batchnorm1(x)
        x = self.relu(self.fc2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
    def predict(self,x):
        output = self.forward(x)
        prediction = t.argmax(output,1)
        return prediction
        


In [149]:
def classifier_train(epochs, model, optimizer, X,y,criterion):
    losses = []
    for i in range(epochs):
        #Precit the output for Given input
        y_pred = model.forward(X)
        #Compute Cross entropy loss
        loss = criterion(y_pred,y)
        #Add loss to the list
        losses.append(loss.item())
        
        #Print loss
        if i%500==0:
            print("Epoch:",i," Loss:",loss.item())
        
        #Clear the previous gradients
        optimizer.zero_grad()
        #Compute gradients
        loss.backward()
        #Adjust weights
        optimizer.step()
    return losses


## Local Training

In [162]:
# Define the input and output
input_channels = X_train_tensor.shape[1]
output_channels = 2

#Initialize the model        
model = Net(input_channels,output_channels)
#Define loss criterion
criterion = t.nn.CrossEntropyLoss()
#Define the optimizer
optimizer = t.optim.Adam(model.parameters(), lr=0.001)
#Number of epochs
epochs = 5000

losses = classifier_train(
    epochs, model, optimizer,X_train_tensor,y_train_tensor,criterion
)

Epoch: 0  Loss: 0.6961112022399902
Epoch: 500  Loss: 0.4703548550605774
Epoch: 1000  Loss: 0.4529033899307251
Epoch: 1500  Loss: 0.4467262923717499
Epoch: 2000  Loss: 0.434055894613266
Epoch: 2500  Loss: 0.4334057569503784
Epoch: 3000  Loss: 0.4298244118690491
Epoch: 3500  Loss: 0.4297804534435272
Epoch: 4000  Loss: 0.4262561500072479
Epoch: 4500  Loss: 0.4146350026130676


## Accuracy on test set

In [165]:
from sklearn.metrics import accuracy_score

def accuracy(model,X,y):
    print(accuracy_score(model.predict(X),y))

In [168]:
accuracy(model,X_test_tensor,y_test_tensor)

0.73
